# Menu Optimizer Flow Demo

This notebook demonstrates the flow of the menu optimization process using the core logic from `menu_solver.py`. It covers:
- Loading a menu request
- Creating the recipe dataset
- Extracting taxonomy constraints
- Solving the menu optimization problem
- Building and displaying the response

## Imports and Setup

Import all necessary modules and set up the environment.

In [ ]:
import pulp
import json
import pandas as pd
from dotenv import load_dotenv, find_dotenv

from menu_optimiser.common import RequestMenu, Args
from menu_optimiser.data import create_dataset
from menu_optimiser.db import get_recipe_ingredients_data
from menu_optimiser.menu_solver import (
    extract_taxonomy_constraints,
    solve_menu_and_storage,
    get_selected_recipes,
    build_response
)
from menu_optimiser.analysis import adjust_available_recipes
from menu_optimiser.config import CompanyOptimizationConfig

In [ ]:
# Load environment variables
load_dotenv(find_dotenv())

## Load Example Menu Request

Load a sample menu request from a JSON file.

In [ ]:
# Path to your test file (adjust as needed)
test_file_path = "../menu_optimiser/test_files/linked/amk_45_link.json"

with open(test_file_path, "r") as f:
    raw_data = json.load(f)
payload = RequestMenu.model_validate(raw_data)

## Create Recipe Dataset

Generate the recipe dataset for the company in the request.

In [ ]:
company = payload.companies[0]
args = Args(company_id=company.company_id, env="prod")
df = await create_dataset(args)
df.head()

In [ ]:
ingredients_df = await get_recipe_ingredients_data()

## Extract Taxonomy Constraints

Extract the taxonomy constraints from the menu request.

In [ ]:
taxonomy_constraints, total_required_recipes, error_msg = extract_taxonomy_constraints(payload)

In [ ]:
print("Total Required Recipes:", total_required_recipes, "\n")

for tax_id, rule in taxonomy_constraints.items():
    print(f"Taxonomy ID: {tax_id}")
    print(f"Total Required Recipes: {rule.total}")
    print(f"Main Ingredients: {rule.main_ingredients}")
    print(f"Price Categories: {rule.price_category}")
    print(f"Cooking Times: {rule.cooking_time_to}")
    print("\n")

## Prepare Available and Required Recipes

Separate available and required recipes for the optimization.

In [ ]:
available_recipes = list(dict.fromkeys(map(int, company.available_recipes)))
required_recipes = list(dict.fromkeys(map(int, company.required_recipes)))

required_recipes_df = pd.DataFrame(df[df["recipe_id"].isin(required_recipes)])
all_available_recipes_df = pd.DataFrame(df[df["recipe_id"].isin(available_recipes)])
available_recipes_df = pd.DataFrame(
    all_available_recipes_df[~all_available_recipes_df["recipe_id"].isin(required_recipes)]
)

print("Required recipes:", required_recipes_df.shape[0])
print("Available recipes for optimization:", available_recipes_df.shape[0])

### If needed, adjust the available recipes

In [ ]:
adjusted_available_recipes, extra_recipes_msg = adjust_available_recipes(
        available_recipe_df=available_recipes_df,
        required_recipe_df=required_recipes_df,
        recipe_universe_df=df,
        taxonomy_constraints=taxonomy_constraints,
    )

In [ ]:
print("Available recipes for optimization:", adjusted_available_recipes.shape[0])
print(extra_recipes_msg)

## Solve the Menu Optimization Problem

Run the optimization to select the best menu according to the constraints.

In [ ]:
company_config = CompanyOptimizationConfig.get_company_config(args.company_id)

In [ ]:
problem, error_messages = await solve_menu_and_storage(
        adjusted_available_recipes,  # available_recipes_df
        required_recipes_df,
        ingredients_df,
        taxonomy_constraints,
        total_required_recipes,
        company_config,
        linked_recipes=company.linked_recipes,
    )

In [ ]:
print("Solver status:", pulp.LpStatus[problem.status])
if error_messages:
    print("Solver messages:", error_messages)

#### Get Selected Recipes

In [ ]:
selected_recipes = get_selected_recipes(problem)
print("Selected recipe IDs:", selected_recipes)

### Build the Menu df

In [ ]:
final_menu_df = pd.DataFrame(
    df[df["recipe_id"].isin(selected_recipes)]
)
final_menu_df.head()

### Build the response

In [ ]:
response = build_response(problem, final_menu_df, payload, error_messages)
print(response)

## Inspecting the problem

In [ ]:
problem.variables()

In [ ]:
problem.constraints

In [ ]:
problem.objective